## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-09-02-24-27 +0000,startribune,"Minnesota Twins, ignited by All-Star Byron Bux...",https://www.startribune.com/minnesota-twins-ch...
1,2025-07-09-02-07-46 +0000,bbc,"Days after Texas floods, at least 161 people a...",https://www.bbc.com/news/articles/cql04vz0q45o
2,2025-07-09-02-02-00 +0000,wsj,China’s Downward Price Pressures Stayed Elevat...,https://www.wsj.com/economy/chinas-downward-pr...
3,2025-07-09-01-51-53 +0000,nyt,Canadian Armed Group Charged in Plot to Seize ...,https://www.nytimes.com/2025/07/08/world/canad...
4,2025-07-09-01-49-09 +0000,nypost,College group Zohran Mamdani co-founded welcom...,https://nypost.com/2025/07/08/us-news/college-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2319/2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
71,trump,26
78,tariffs,8
321,tariff,5
154,new,5
11,texas,5


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
8,2025-07-09-01-11-00 +0000,wsj,President Trump decided to delay the implement...,https://www.wsj.com/economy/trade/trump-tariff...,58
31,2025-07-08-21-38-00 +0000,wsj,Businesses hoping for clarity on where Trump t...,https://www.wsj.com/economy/trade/trump-tariff...,55
72,2025-07-08-13-59-19 +0000,nyt,Trump’s New Trade Threats Set Off Global Scram...,https://www.nytimes.com/2025/07/08/business/ec...,51
74,2025-07-08-13-44-50 +0000,nyt,Trump Targets Japan With 25% Tariffs After Sev...,https://www.nytimes.com/2025/07/08/business/tr...,47
38,2025-07-08-20-58-00 +0000,wsj,Copper prices surged and the Dow industrials s...,https://www.wsj.com/economy/trade/trumps-tarif...,47


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
8,58,2025-07-09-01-11-00 +0000,wsj,President Trump decided to delay the implement...,https://www.wsj.com/economy/trade/trump-tariff...
9,24,2025-07-09-01-00-00 +0000,wsj,Two Republicans named Kevin are vying to be th...,https://www.wsj.com/economy/central-banking/tr...
4,23,2025-07-09-01-49-09 +0000,nypost,College group Zohran Mamdani co-founded welcom...,https://nypost.com/2025/07/08/us-news/college-...
16,22,2025-07-08-23-26-50 +0000,nypost,Mayor Eric Adams builds out re-election campai...,https://nypost.com/2025/07/08/us-news/mayor-er...
17,21,2025-07-08-23-16-20 +0000,nypost,Sen. Ted Cruz pictured at Parthenon on Greek v...,https://nypost.com/2025/07/08/us-news/ted-cruz...
31,21,2025-07-08-21-38-00 +0000,wsj,Businesses hoping for clarity on where Trump t...,https://www.wsj.com/economy/trade/trump-tariff...
42,21,2025-07-08-20-06-40 +0000,startribune,Minnesota Vikings player lost $240K in bank fr...,https://www.startribune.com/minnesota-vikings-...
24,20,2025-07-08-22-42-03 +0000,nypost,Gavin Newsom says he’s ‘spellbound’ by AME bis...,https://nypost.com/2025/07/08/us-news/gavin-ne...
47,20,2025-07-08-19-28-44 +0000,nypost,Zelensky’s right-hand man thanks Trump for rev...,https://nypost.com/2025/07/08/us-news/zelensky...
38,18,2025-07-08-20-58-00 +0000,wsj,Copper prices surged and the Dow industrials s...,https://www.wsj.com/economy/trade/trumps-tarif...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
